In [1]:
import numpy as np
import pandas as pd

import pickle
import random
import os
import time
from datetime import datetime
from pathlib import Path
import glob
import json
from tqdm.notebook import tqdm

from multiprocessing import Pool, cpu_count
from post_multi import motion_multi, fs_multi, fs_multiF

import matplotlib.path as mpltPath
from scipy.spatial import distance
from scipy.optimize import minimize
from scipy.interpolate import interp1d


import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from PIL import Image
#%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)
sns.set_style('darkgrid')

from xyz10.io_f_mod import read_data_file
from xyz10.visualize_f_mod import visualize_trajectory, save_figure_to_image
from xyz10.compute_f_mod import compute_step_positions, compute_step_positions_mod, split_ts_seq
from xyz10.compute_f_mod import compute_steps, compute_headings, compute_stride_length, compute_step_heading, compute_rel_positions
from xyz10.compute_f_mod import correct_positions,  correct_positions_mod

from xyz10.io_f_mod import read_data_file
from xyz10.visualize_f_mod import visualize_trajectory, save_figure_to_image

In [19]:
import glob, os
paths = glob.glob('./img_out/predictions/blend_x7_0305/*/*snap2motion4s.png')
_ = [os.remove(path) for path in paths]

---------------------------------------------------------

Function: Blend, Plot_Traces, Plot_Grid, Make_Submit

In [2]:
FLOOR_NUM_to_ID = {'5a0546857ecc773753327266': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5c3c44b80379370013e0fd2b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5d27075f03f801723c2e360f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                 '5d27096c03f801723c31e5e0': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6'},
                 '5d27097f03f801723c320d97': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5d27099f03f801723c32511d': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5d2709a003f801723c3251bf': {0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                 '5d2709b303f801723c327472': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                 '5d2709bb03f801723c32852c': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5d2709c303f801723c3299ee': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F', 4: '5F', 5: '6F', 6: '7F', 7: '8F', 8: '9F'},
                 '5d2709d403f801723c32bd39': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F'},
                 '5d2709e003f801723c32d896': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da138274db8ce0c98bbd3d2': {0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da1382d4db8ce0c98bbe92e': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da138314db8ce0c98bbf3a0': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da138364db8ce0c98bc00f1': {0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da1383b4db8ce0c98bc11ab': {0: 'F1', 1: 'F2', 2: 'F3'},
                 '5da138754db8ce0c98bca82f': {0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5da138764db8ce0c98bcaa46': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da1389e4db8ce0c98bd0547': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                 '5da138b74db8ce0c98bd4774': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                 '5da958dd46f8266d0737457b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                 '5dbc1d84c1eb61796cf7c010': {-1: 'B1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8'},
                 '5dc8cea7659e181adb076a3f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'}}

def blend_predictions(blend_folder, reference_file):

    blend_paths = glob.glob(blend_folder + "*")
    blend_data = []
    for b_path in blend_paths:
        blend_data.append(pickle.load(open(b_path, "rb")))
        #break
    reference_data = pickle.load(open(blend_folder+reference_file, "rb"))

    compound= {}
    for site_id in reference_data.keys():

        compound[site_id] = {}
        for trace_id in reference_data[site_id].keys():

            _timestamps = reference_data[site_id][trace_id].to_numpy()[:, 3]
            _floor = int(np.median(reference_data[site_id][trace_id].to_numpy()[:, 2]))
            _x = []
            _y = []
            for data in blend_data:

                predicted_record = data[site_id][trace_id].to_numpy()
                _x.append(interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")(_timestamps))
                _y.append(interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")(_timestamps))

            compound[site_id][trace_id] = pd.DataFrame({"x": np.median(_x, axis=0), "y": np.median(_y, axis=0), "floor": _floor, "time": _timestamps})

            #break 
    return compound

def make_submission_bkp(model_name, data, sufix="coarse"):

    sample_submit = pd.read_csv("./submit/sample_submission.csv")
    splits = sample_submit.site_path_timestamp.str.split(pat="_", expand=True)
    sub_data = sample_submit.copy(deep=True).join(splits)
    sub_data.rename(columns={0:"site", 1:"path", 2:"timestamp"}, inplace=True)

    for i in tqdm(list(sub_data.index)):
        site_id = sub_data.site[i]
        trace_id = sub_data.path[i]
        timestamp = sub_data.timestamp[i]

        predicted_record = data[site_id][trace_id].to_numpy()

        func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
        func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

        sub_data.loc[i, "x"] = func_x(timestamp)
        sub_data.loc[i, "y"] = func_y(timestamp)
        sub_data.loc[i, "floor"] = int(np.median(predicted_record[:, 2]))
        #break

    _ = [sub_data.pop(col) for col in ["site", "path", "timestamp"]]

    sub_data.to_csv(f"./submit/{model_name}_{sufix}.csv", index=False)
    

def make_submission(model_name, data, sufix="coarse"):
    
    sample_submit = pd.read_csv("./submit/sample_submission.csv")
    splits = sample_submit.site_path_timestamp.str.split(pat="_", expand=True)
    sub_data = sample_submit.copy(deep=True).join(splits)
    sub_data.rename(columns={0:"site", 1:"path", 2:"timestamp"}, inplace=True)

    gr = sub_data.groupby("path")
    for trace_id in gr.groups:
        timestamps = sub_data.loc[gr.groups[trace_id]].timestamp.to_list()

        site_id = sub_data.loc[gr.groups[trace_id]].site.to_list()[0]
        predicted_record = data[site_id][trace_id].to_numpy()

        func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
        func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

        sub_data.loc[gr.groups[trace_id], "x"] = func_x(timestamps)
        sub_data.loc[gr.groups[trace_id], "y"] = func_y(timestamps)
        sub_data.loc[gr.groups[trace_id], "floor"] = predicted_record[0, 2]
        #break

    _ = [sub_data.pop(col) for col in ["site", "path", "timestamp"]]

    sub_data.to_csv(f"./submit/{model_name}_{sufix}.csv", index=False)
    
def plot_predictions_multi(model_name, data, sufix="coarse", delay_suffix=False):
    
    def swap_trace_floor(predicted_data):
        swap = {}

        for site_id in predicted_data.keys():

            swap[site_id] = {}
            for trace_id in predicted_data[site_id].keys():

                floor_id = predicted_data[site_id][trace_id].floor[0]
                if floor_id not in swap[site_id].keys():
                    swap[site_id][floor_id] = {}
                swap[site_id][floor_id][trace_id] = predicted_data[site_id][trace_id]

        return swap

    data = swap_trace_floor(data)

    try:
        os.makedirs(f"./img_out/predictions/{model_name}/")
    except:
        pass

    n_s = 0
    for site_id in tqdm(data.keys()):  # over sites 
        n_s += 1
        #print(f"Processing Trajectories #{n_s}: Site-{site_id} with {len(data[site_id])} traces")

        try:
            os.makedirs(f"./img_out/predictions/{model_name}/{site_id}")
        except:
            pass

        for floor_id in data[site_id]:  # over traces
            site_path = "./data_in/metadata/" + site_id + "/"
            
            positions = []
            legends = []
            for trace_id in data[site_id][floor_id].keys():
                positions.append(data[site_id][floor_id][trace_id].to_numpy()[:, :2])
                
                if delay_suffix:
                    legends.append(f"{trace_id}_s{int(0.001*data[site_id][floor_id][trace_id].to_numpy()[0, 4])}_e{int(0.001*data[site_id][floor_id][trace_id].to_numpy()[1, 4])}")
                else:
                    legends.append(trace_id)

            try:
                floor = FLOOR_NUM_to_ID[site_id][floor_id]

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                with open(info_path) as info_file:
                    info_data = json.load(info_file)             

                map_width = info_data["map_info"]["width"]
                map_height = info_data["map_info"]["height"]

                fig_steps = visualize_trajectory(trajectory=positions, is_multi = True,
                                                 floor_plan_filename=map_path, mode="lines + markers", title=f"{site_id}_{floor}_{sufix}", legends=legends, 
                                                 width_meter=map_width,  height_meter=map_height)
                save_figure_to_image(fig_steps, f"./img_out/predictions/{model_name}/{site_id}/{floor}_{sufix}.png")
            except:
                print(f"Exception: wrong floor-{floor} site-{site_id}")

        #break  # only first site_id
        
def plot_grid(model_name, data, sufix="coarse", delay_suffix=False):

    try:
        os.makedirs(f"./img_out/{model_name}/")
    except:
        pass

    n_s = 0
    for site_id in tqdm(data.keys()):  # over sites 
        n_s += 1
        #print(f"Processing Trajectories #{n_s}: Site-{site_id} with {len(data[site_id])} traces")

        try:
            os.makedirs(f"./img_out/{model_name}/{site_id}")
        except:
            pass

        for floor_id in data[site_id]:  # over traces
            site_path = "./data_in/metadata/" + site_id + "/"
            
            positions = data[site_id][floor_id]
            legends = []

            try:
                floor = FLOOR_NUM_to_ID[site_id][floor_id]

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                with open(info_path) as info_file:
                    info_data = json.load(info_file)             

                map_width = info_data["map_info"]["width"]
                map_height = info_data["map_info"]["height"]

                fig_steps = visualize_trajectory(trajectory=positions, is_multi = False,
                                                 floor_plan_filename=map_path, mode="markers", title=f"{site_id}_{floor}_{sufix}", legends=legends, 
                                                 width_meter=map_width,  height_meter=map_height)
                save_figure_to_image(fig_steps, f"./img_out/{model_name}/{site_id}/{floor}_{sufix}.png")
            except:
                print(f"Exception: wrong floor-{floor} site-{site_id}")
                
        #break


In [3]:
def snap2grid(predicted_data, grid_siteid_floorid, timestamps_traceid, snap_range=5):

    def closest_point(path, point, snap_range):
        #print(path)
        distance = (path[:, 0] - point[0])**2 + (path[:, 1] - point[1])**2
        
        if distance.min() < snap_range**2:
            idx = distance.argmin()
            return [path[idx, 0], path[idx, 1]]
        else:
            return [point[0], point[1]]

    snap2grid_data = {}

    n_s= 0
    for site_id in tqdm(predicted_data.keys()):  # over sites
        n_s += 1

        snap2grid_data[site_id] = {}
        for trace_id in predicted_data[site_id].keys():  # over traces

            trace = []  # list of points [x,y]
            predicted_record = predicted_data[site_id][trace_id].to_numpy()
            
            floor_id = int(np.median(predicted_record[:, 2]))
            grid = grid_siteid_floorid[site_id][floor_id]
            
            func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
            func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

            _x = func_x(timestamps_traceid[trace_id])
            _y = func_y(timestamps_traceid[trace_id])

            for i, _ in enumerate(_x):  # over points 
                point = [_x[i], _y[i]]
                trace.append(closest_point(grid, point, snap_range))  # grid = closest step/waypoint point(slow) vs path_a0= closest contour point (fast) 

            snap2grid_data[site_id][trace_id] = pd.DataFrame(trace, columns=["x", "y"])
            snap2grid_data[site_id][trace_id]["floor"] = floor_id
            snap2grid_data[site_id][trace_id]["time"] = timestamps_traceid[trace_id]
            
    return snap2grid_data

def snap2grid_v2(predicted_data, grid_siteid_floorid, aux_grid_siteid_floorid, timestamps_traceid, snap_range=5, aux_snap_range=30):

    def closest_point(path, point, snap_range):
        #print(path)
        distance = (path[:, 0] - point[0])**2 + (path[:, 1] - point[1])**2
        
        if distance.min() < snap_range**2:
            idx = distance.argmin()
            return [path[idx, 0], path[idx, 1]], True
        else:
            return [point[0], point[1]], False

    snap2grid_data = {}

    n_s= 0
    for site_id in tqdm(predicted_data.keys()):  # over sites
        n_s += 1

        snap2grid_data[site_id] = {}
        for trace_id in predicted_data[site_id].keys():  # over traces

            trace = []  # list of points [x,y]
            predicted_record = predicted_data[site_id][trace_id].to_numpy()
            
            floor_id = int(np.median(predicted_record[:, 2]))
            grid = grid_siteid_floorid[site_id][floor_id]
            
            func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
            func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

            _x = func_x(timestamps_traceid[trace_id])
            _y = func_y(timestamps_traceid[trace_id])

            for i, _ in enumerate(_x):  # over points 
                point = [_x[i], _y[i]]
                _closest_point, _isOk = closest_point(grid, point, snap_range)
                if _isOk:
                    trace.append(_closest_point)  # grid = closest step/waypoint point(slow) vs path_a0= closest contour point (fast)
                else:
                    _aus_close_point, _ = closest_point(aux_grid, point, aux_snap_range)
                    trace.append(_aus_close_point)

            snap2grid_data[site_id][trace_id] = pd.DataFrame(trace, columns=["x", "y"])
            snap2grid_data[site_id][trace_id]["floor"] = floor_id
            snap2grid_data[site_id][trace_id]["time"] = timestamps_traceid[trace_id]
            
    return snap2grid_data

---------------------------------------------------------

Snap2Motion

In [4]:
DELAY_OFFSET = 500 # msecs
V_LIMIT = 0.0015  # m/msec
INVALID_RANGE = 0.5 # invalidity distance in fraction of trajectory extent

def optimize_trajectory_motion(motion_x, motion_y, motion_t,
                               coarse_x, coarse_y, coarse_t,
                               leaked_record, step_range, angle_range, angle0_range,
                               verbose=False):  # +-step_range in fraction of step length; +-angle_range in fraction of Pi

    num_elements = motion_t.shape[0]
    trajectory_extent = ((coarse_x.max() - coarse_x.min()) ** 2 + (coarse_x.max() - coarse_x.min()) ** 2) ** 0.5

    time_deltas = np.diff(motion_t)  # size "n-1"
    motion_steps_lengths_input = ((motion_x ** 2 + motion_y ** 2) ** 0.5)[1:]  # size "n-1"
    motion_angles_input = np.arctan2(motion_y[1:], motion_x[1:])  # size "n-1"
    # print(num_elements)

    ################  INITIAL/BOUNDARIES VALUES   ##################################
    step_lengths_ini = np.amin(np.vstack([time_deltas * V_LIMIT, motion_steps_lengths_input]), axis=0)
    angles_ini = motion_angles_input

    step_lengths_min = np.amin(np.vstack([time_deltas * V_LIMIT * 0.99, motion_steps_lengths_input * (1 - step_range)]),
                               axis=0)
    step_lengths_max = np.amin(np.vstack([time_deltas * V_LIMIT * 1.00, motion_steps_lengths_input * (1 + step_range)]),
                               axis=0)  # not more than maximum speed nor allowed variation of step length
    angles_min = motion_angles_input - np.pi * angle_range
    angles_max = motion_angles_input + np.pi * angle_range

    angle0_ini = np.array([0.0])

    # if non sequential=>negative-invalid delay/negative-invalid position point/delay is too long => take no limits, i.e. something large e.g. 1e6 = 1000s ~ 1500meters
    # make rough estimate based on median of coarse_data in case of very long/invalid delays/invaliude start-end points(-1m value)

    startpoint_invalid = False
    if leaked_record["start_x"] > 0 and leaked_record["start_delay"] > 0 and (
            V_LIMIT * leaked_record["start_delay"]) < trajectory_extent * INVALID_RANGE:
        _start_delay = leaked_record["start_delay"] - DELAY_OFFSET if leaked_record[
                                                                          "start_delay"] > DELAY_OFFSET else 100
    else:
        startpoint_invalid = True
        _start_delay = 1e6

    endpoint_invalid = False
    if leaked_record["end_x"] > 0 and leaked_record["end_delay"] > 0 and (
            V_LIMIT * leaked_record["end_delay"]) < trajectory_extent * INVALID_RANGE:
        _end_delay = leaked_record["end_delay"] - DELAY_OFFSET if leaked_record["end_delay"] > DELAY_OFFSET else 100

    else:
        endpoint_invalid = True
        _end_delay = 1e6

    initial_start = np.array([coarse_x[0], coarse_y[0]])
    initial_end = np.array([coarse_x[-1], coarse_y[-1]])

    x_start_min = leaked_record["start_x"] - _start_delay * V_LIMIT
    x_start_max = leaked_record["start_x"] + _start_delay * V_LIMIT
    x_end_min = leaked_record["end_x"] - _end_delay * V_LIMIT
    x_end_max = leaked_record["end_x"] + _end_delay * V_LIMIT
    y_start_min = leaked_record["start_y"] - _start_delay * V_LIMIT
    y_start_max = leaked_record["start_y"] + _start_delay * V_LIMIT
    y_end_min = leaked_record["end_y"] - _end_delay * V_LIMIT
    y_end_max = leaked_record["end_y"] + _end_delay * V_LIMIT

    bounds_angle0 = [(-angle0_range * np.pi, angle0_range * np.pi)]
    bounds_start = [(0, None), (0, None)]
    # bounds_start = [(x_start_min, x_start_max), (y_start_min, y_start_max)]  # 2x pairs
    bounds_end = [(0, None), (0, None)]
    # bounds_end = [(x_end_min, x_end_max), (y_end_min, y_end_max)] # 2x pairs
    bounds_steps = [(min_el, max_el) for min_el, max_el in zip(step_lengths_min, step_lengths_max)]  # (n-1)x pairs
    bounds_angles = [(min_el, max_el) for min_el, max_el in zip(angles_min, angles_max)]  # (n-1)x pairs

    if endpoint_invalid:
        initial_guess = np.concatenate([step_lengths_ini, angles_ini, angle0_ini, initial_start])  # 1+(2*n+2)x
        bounds = bounds_steps + bounds_angles + bounds_angle0 + bounds_start  # 1+(2*n+2)x pairs
    else:
        initial_guess = np.concatenate(
            [step_lengths_ini, angles_ini, angle0_ini, initial_start, initial_end])  # 1+(2*n+2)x
        bounds = bounds_steps + bounds_angles + bounds_angle0 + bounds_start + bounds_end  # 1+(2*n+2)x pairs

    if verbose:
        print("----------------------------------------------------------")
        print("invalid distance", trajectory_extent * INVALID_RANGE)
        print(f"invalid start/end: {startpoint_invalid}/{endpoint_invalid}")
        print(
            f'leaked_start/end:  {leaked_record["start_x"]}-{leaked_record["start_y"]}/{leaked_record["end_x"]}-{leaked_record["end_y"]}')
        print(f"_start/end_delay: {_start_delay}/{_end_delay} ")
        print(f"initial_start/end:  {initial_start}/{initial_end}")
        print(f"bounds_start/end: {bounds_start}/{bounds_end}")
        print("----------------------------------------------------------")
    ###############################################################################
    d_ind_a = 1  # indices shift due to alpha0

    def loss(params):
        # print(params.shape)
        _d_xs = np.cumsum(params[: num_elements - 1] * np.cos(
            params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
        _d_ys = np.cumsum(params[: num_elements - 1] * np.sin(
            params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
        _xs = np.hstack([params[2 * num_elements - 2 + d_ind_a + 0],
                         params[2 * num_elements - 2 + d_ind_a + 0] + _d_xs])
        _ys = np.hstack([params[2 * num_elements - 2 + d_ind_a + 1],
                         params[2 * num_elements - 2 + d_ind_a + 1] + _d_ys])

        if not endpoint_invalid:
            _xs = _xs - (_xs[-1] - params[-2]) / 1
            _ys = _ys - (_ys[-1] - params[-1]) / 1

        _loss = np.sum(((coarse_x - _xs) ** 2 + (coarse_y - _ys) ** 2) ** 0.5) / num_elements  # MAE
        return _loss

    results = minimize(fun=loss,
                       x0=initial_guess,
                       bounds=bounds,
                       options={'maxcor': 50, 'ftol': 1e-09, 'gtol': 1e-09, 'maxfun': 40000, 'maxiter': 40000,
                                'maxls': 50},
                       method="L-BFGS-B")
    params = results.x

    _d_xs = np.cumsum(params[: num_elements - 1] * np.cos(
        params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
    _d_ys = np.cumsum(params[: num_elements - 1] * np.sin(
        params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
    _xs = np.hstack([params[2 * num_elements - 2 + d_ind_a + 0],
                     params[2 * num_elements - 2 + d_ind_a + 0] + _d_xs])
    _ys = np.hstack([params[2 * num_elements - 2 + d_ind_a + 1],
                     params[2 * num_elements - 2 + d_ind_a + 1] + _d_ys])

    if not endpoint_invalid:
        _xs = _xs - (_xs[-1] - params[-2]) / 1
        _ys = _ys - (_ys[-1] - params[-1]) / 1

    return _xs, _ys, results.fun

############################################################################################################
def snap2motion(coarse_data, motion_data, leaked_data, step_range, angle_range, angle0_range):

    snapped_data = {}
    site_ids =  ["5da1389e4db8ce0c98bd0547", "5d27075f03f801723c2e360f", "5d2709b303f801723c327472", "5d27097f03f801723c320d97", "5da138b74db8ce0c98bd4774", "5d2709d403f801723c32bd39"]
    #site_ids = coarse_data.keys()

    for site_id in tqdm(site_ids):
        
        #print(site_id)
        
        snapped_data[site_id] = {}
        for trace_id in tqdm(coarse_data[site_id]):  # over traces
            
            
            verbose_ids = ["9cc4412ff73ec37e30be9d9f", "472be94f5be907c04c932114", "4ff5b9152353b3dc458992fa", "4affd104e0ec7a7806edfa78"]
            verbose = trace_id in verbose_ids
            #verbose = True
            if verbose:
                print(trace_id)
            
            coarse_record = coarse_data[site_id][trace_id][["time", "x", "y"]].copy(deep=True)
            motion_record = motion_data[site_id][trace_id][["time", "rx", "ry"]].copy(deep=True)
            leaked_record = leaked_data[site_id][trace_id]
              
            _x, _y, _fun = optimize_trajectory_motion(motion_record.rx.to_numpy(),
                                                      motion_record.ry.to_numpy(),
                                                      motion_record.time.to_numpy(),  # sensor data (relative steps vs cumulative)
                                                      coarse_record.x.to_numpy(),
                                                      coarse_record.y.to_numpy(),
                                                      coarse_record.time.to_numpy(),  # wifi data
                                                      leaked_record,
                                                      step_range, angle_range, angle0_range, verbose=verbose)
            
            snapped_data[site_id][trace_id] = pd.DataFrame(_x, columns=["x"])
            snapped_data[site_id][trace_id]["y"] = _y
            snapped_data[site_id][trace_id]["floor"] = leaked_record["floor"]
            snapped_data[site_id][trace_id]["time"] = motion_record.time.to_numpy()
            
            #print(_fun)
            #break
        #break
        
    return snapped_data

###############    MULTIPROCESSING VERSION     #################################################################
def snap2motion_multi(coarse_data, motion_data, leaked_data, step_range, angle_range, angle0_range):

    site_ids = ["5d2709b303f801723c327472"]#["5da1389e4db8ce0c98bd0547", "5d27075f03f801723c2e360f", "5d2709b303f801723c327472", "5d27097f03f801723c320d97", "5da138b74db8ce0c98bd4774", "5d2709d403f801723c32bd39"]
    site_ids = coarse_data.keys()
    
    input_data = [(site_id, coarse_data[site_id], motion_data[site_id], leaked_data[site_id], step_range, angle_range, angle0_range) for site_id in site_ids]
    
    pool = Pool(cpu_count()-3) # Create a multiprocessing Pool
    snapped_data = pool.starmap(motion_multi, input_data)  # process input_data iterable with pool

    pool.close()
    pool.join()
        
    res_d = snapped_data[0]
    for i, _d in enumerate(snapped_data):
        if i>0:
            res_d.update(_d)
        
    return res_d

In [11]:
model_name = "blend_x7_0305"
parsed_test_data = pickle.load(open("./data_out/full24/test-10k_mix-counts_t550.pkl", "rb"))   # counts_t550 vs counts
predicted_data = pickle.load(open("./submit/fit_data/blend_x7_0305.pkl", "rb"))   #
leaked_data = pickle.load(open("./data_out/leaked/leaked_siteid_traceid_all_delays.pkl", "rb")) 

snapped2motion = snap2motion_multi(predicted_data, parsed_test_data, leaked_data, step_range=0.3, angle_range=0.0001, angle0_range=0.0001)  # v2s

In [12]:
plot_predictions_multi(model_name, snapped2motion, sufix=f"snap2motion2s")

  0%|          | 0/24 [00:00<?, ?it/s]

In [13]:
with open(f"./submit/fit_data/post/{model_name}_snapped2motion2s.pkl", "wb") as f:
    pickle.dump(snapped2motion, f)

---------------------------------------------------------

Snap2FS

In [4]:
V_LIMIT = 0.0015  # m/msec
INVALID_RANGE = 0.5 # invalidity distance in fraction of trajectory extent
SCALE_XY0 = 0.01

In [5]:
def optimize_trajectory_fs_v1(motion_x, motion_y, motion_t,
                              fs_points,
                              leaked_record,
                              snap_range, # +-snap/start_point shift range in fraction of x/y bounding box (max-min)
                              step_range, angle_range, angle0_range):  # +-step_range in fraction of step length; +-angle_range in fraction of Pi
    
    num_elements = motion_t.shape[0]
    trajectory_extent = ((motion_x.max()-motion_x.min())**2 + (motion_y.max()-motion_y.min())**2)**0.5
    trajectory_extent_x = abs(motion_x.max()-motion_x.min())   
    trajectory_extent_y = abs(motion_y.max()-motion_y.min())
    
    snap_distance = snap_range * trajectory_extent
    snap_distance_x = snap_range * trajectory_extent_x
    snap_distance_y = snap_range * trajectory_extent_y
    
    time_deltas = np.diff(motion_t)  # size "n-1"
    motion_dx = np.diff(motion_x)
    motion_dy = np.diff(motion_y)
    
    motion_steps_lengths_input = (motion_dx**2 + motion_dy**2)**0.5
    motion_angles_input = np.arctan2(motion_dy, motion_dx) 
    
    d_ind_a = 1  # index shift due to angle0 parameter
    #print(num_elements)

    ################  INITIAL/BOUNDARIES VALUES   ##################################      
    step_lengths_ini = np.amin(np.vstack([time_deltas * V_LIMIT, motion_steps_lengths_input]), axis=0)
    angles_ini = motion_angles_input
    
    step_lengths_min = np.amin(np.vstack([time_deltas * V_LIMIT * 0.99, motion_steps_lengths_input*(1-step_range)]), axis=0)
    step_lengths_max = np.amin(np.vstack([time_deltas * V_LIMIT * 1.00, motion_steps_lengths_input*(1+step_range)]), axis=0)  # not more than maximum speed nor allowed variation of step length
    angles_min = motion_angles_input - np.pi*angle_range
    angles_max = motion_angles_input + np.pi*angle_range
    
    angle0_ini = np.array([0.0])
  
    initial_start = np.array([motion_x[0], motion_y[0]])

    x_start_min = initial_start[0] - snap_distance_x
    x_start_max = initial_start[0] + snap_distance_x
    y_start_min = initial_start[1] - snap_distance_y
    y_start_max = initial_start[1] + snap_distance_y
    
    bounds_angle0 = [(-angle0_range*np.pi, angle0_range*np.pi)]
    bounds_start = [(0, None), (0, None)]#[(x_start_min, x_start_max), (y_start_min, y_start_max)]  # 2x pairs
    bounds_steps = [(min_el, max_el) for min_el, max_el in zip(step_lengths_min, step_lengths_max)] # (n-1)x pairs
    bounds_angles = [(min_el, max_el) for min_el, max_el in zip(angles_min, angles_max)] # (n-1)x pairs
    
    initial_guess =  np.concatenate([step_lengths_ini, angles_ini, angle0_ini, initial_start])# (2*n+2)x
    bounds = bounds_steps + bounds_angles + bounds_angle0 + bounds_start # (2*n+2)x pairs
    ###############################################################################
    #########################  SNAPPING POINTS   ##################################
    snap_points = []
    
    for i_step, _ in enumerate(motion_x):
        _distances = ((fs_points[:, 0] - motion_x[i_step])**2 + (fs_points[:, 1] - motion_y[i_step])**2)**0.5
        _snap_points = fs_points[_distances <= snap_distance]
        
        if len(_snap_points) > 0:
            snap_points.append(_snap_points)
                
    snap_points = np.concatenate(snap_points)
    snap_points = np.array(list(set(map(tuple, snap_points))))
    
    #print("snap_distance", snap_distance)
    #print("snap_points.shape", snap_points.shape)
    ##################################################################################
    ######################### OPTIMIZE ###############################################
    if len(snap_points) > 0:  # some points are within snapping range
               
        def loss(params):   
            #print(params.shape)
            _d_xs = np.cumsum(params[: num_elements-1] * np.cos(params[2*num_elements-2]+params[num_elements-1: 2*num_elements-2]))
            _d_ys = np.cumsum(params[: num_elements-1] * np.sin(params[2*num_elements-2]+params[num_elements-1: 2*num_elements-2]))
            _xs = np.hstack([params[2*num_elements-2 + d_ind_a + 0], 
                             params[2*num_elements-2 + d_ind_a + 0] + _d_xs])
            _ys = np.hstack([params[2*num_elements-2 + d_ind_a + 1], 
                             params[2*num_elements-2 + d_ind_a + 1] + _d_ys])

            distances2 = (snap_points[:, 0].reshape(-1,1) - _xs.reshape(1,-1))**2 + (snap_points[:, 1].reshape(-1,1) - _ys.reshape(1,-1))**2
            _loss = np.sum(np.amin(distances2, axis=0)**0.5)  # near-field
            #_loss = np.sum(np.amin(distances2**0.5, axis=1))  # MAE
            #_loss = np.sum(np.amin(distances2, axis=1))  # MSE

            return _loss/num_elements

        results = minimize(fun=loss,
                           x0=initial_guess,
                           bounds=bounds,
                           method="L-BFGS-B")

        params = results.x

        _d_xs = np.cumsum(params[: num_elements-1] * np.cos(params[2*num_elements-2]+params[num_elements-1: 2*num_elements-2]))
        _d_ys = np.cumsum(params[: num_elements-1] * np.sin(params[2*num_elements-2]+params[num_elements-1: 2*num_elements-2]))
        _xs = np.hstack([params[2*num_elements-2 + d_ind_a + 0], 
                         params[2*num_elements-2 + d_ind_a + 0] + _d_xs])
        _ys = np.hstack([params[2*num_elements-2 + d_ind_a + 1], 
                         params[2*num_elements-2 + d_ind_a + 1] + _d_ys])

        #print("done fitting")
        return _xs, _ys, results.fun
    else:
        return motion_x, motion_y, -1

In [6]:
def optimize_trajectory_fs_v2(motion_x, motion_y, motion_t,
                              fs_points,
                              leaked_record,
                              snap_range,  # +-snap/start_point shift range in fraction of x/y bounding box (max-min)
                              step_range, angle_range,
                              angle0_range):  # +-step_range in fraction of step length; +-angle_range in fraction of Pi

    # REVERSE START <-> END POINTS?
    is_start_semivalid = leaked_record["start_x"] > 0 and leaked_record["start_delay"] > 0
    is_end_semivalid = leaked_record["end_x"] > 0 and leaked_record["end_delay"] > 0

    if is_start_semivalid and is_end_semivalid:
        is_reversed = leaked_record["start_delay"] > leaked_record["end_delay"]
    elif is_end_semivalid:
        is_reversed = True
    else:
        is_reversed = False

    leaked_delay = leaked_record["end_delay"] if is_reversed else leaked_record["start_delay"]
    leaked_x = leaked_record["end_x"] if is_reversed else leaked_record["start_x"]
    leaked_y = leaked_record["end_y"] if is_reversed else leaked_record["start_y"]

    if is_reversed:
        motion_x = motion_x[::-1]
        motion_y = motion_y[::-1]
        motion_t = motion_t[::-1]
    ###################################################################
    num_elements = motion_t.shape[0]
    trajectory_extent = ((motion_x.max() - motion_x.min()) ** 2 + (motion_y.max() - motion_y.min()) ** 2) ** 0.5
    trajectory_extent_x = abs(motion_x.max() - motion_x.min())
    trajectory_extent_y = abs(motion_y.max() - motion_y.min())

    snap_distance = snap_range * trajectory_extent
    snap_distance_x = snap_range * trajectory_extent_x
    snap_distance_y = snap_range * trajectory_extent_y

    time_deltas = np.abs(np.diff(motion_t))  # size "n-1"
    motion_dx = np.diff(motion_x)
    motion_dy = np.diff(motion_y)

    motion_steps_lengths_input = (motion_dx ** 2 + motion_dy ** 2) ** 0.5
    motion_angles_input = np.arctan2(motion_dy, motion_dx)
    # print(num_elements)
    ################  INITIAL/BOUNDARIES VALUES   ##################################
    step_lengths_ini = np.amin(np.vstack([time_deltas * V_LIMIT, motion_steps_lengths_input]), axis=0)
    angles_ini = motion_angles_input

    step_lengths_min = np.amin(np.vstack([time_deltas * V_LIMIT * 0.99, motion_steps_lengths_input * (1 - step_range)]),
                               axis=0)
    step_lengths_max = np.amin(np.vstack([time_deltas * V_LIMIT * 1.00, motion_steps_lengths_input * (1 + step_range)]),
                               axis=0)  # not more than maximum speed nor allowed variation of step length
    angles_min = motion_angles_input - np.pi * angle_range
    angles_max = motion_angles_input + np.pi * angle_range

    angle0_ini = np.array([0.0])

    initial_start = np.array([motion_x[0], motion_y[0]])

    x_start_min = initial_start[0] - snap_distance_x
    x_start_max = initial_start[0] + snap_distance_x
    y_start_min = initial_start[1] - snap_distance_y
    y_start_max = initial_start[1] + snap_distance_y
    ############# Limit snap range by leaked data (if narrower and leaked data valid)  #######
    if leaked_x > 0 and leaked_delay > 0 and (V_LIMIT * leaked_delay) < trajectory_extent * INVALID_RANGE:
        _x_start_min = max(leaked_x - V_LIMIT * leaked_delay, x_start_min)
        _x_start_max = min(leaked_x + V_LIMIT * leaked_delay, x_start_max)
        _y_start_min = max(leaked_y - V_LIMIT * leaked_delay, y_start_min)
        _y_start_max = min(leaked_y + V_LIMIT * leaked_delay, y_start_max)

        x_start_min, x_start_max = (_x_start_min, _x_start_max) if _x_start_min < _x_start_max else (
        x_start_min, x_start_max)
        y_start_min, y_start_max = (_y_start_min, _y_start_max) if _y_start_min < _y_start_max else (
        y_start_min, y_start_max)

        #initial_start = np.array([(x_start_min + x_start_max) / 2, (y_start_min + y_start_max) / 2])
    ################ RESCALE XY0/INI-BOUNDS  ####################
    initial_start *= SCALE_XY0
    x_start_min *= SCALE_XY0
    x_start_max *= SCALE_XY0
    y_start_min *= SCALE_XY0
    y_start_max *= SCALE_XY0
    #################################################################
    bounds_angle0 = [(-angle0_range * np.pi, angle0_range * np.pi)]
    bounds_start = [(0, None), (0, None)]
    #bounds_start = [(x_start_min, x_start_max), (y_start_min, y_start_max)]  # 2x pairs
    bounds_steps = [(min_el, max_el) for min_el, max_el in zip(step_lengths_min, step_lengths_max)]  # (n-1)x pairs
    bounds_angles = [(min_el, max_el) for min_el, max_el in zip(angles_min, angles_max)]  # (n-1)x pairs

    initial_guess = np.concatenate([step_lengths_ini, angles_ini, angle0_ini, initial_start])  # (2*n+2)x
    bounds = bounds_steps + bounds_angles + bounds_angle0 + bounds_start  # (2*n+2)x pairs
    ###############################################################################
    #########################  SNAPPING POINTS   ##################################
    snap_points = []

    for i_step, _ in enumerate(motion_x):
        _distances = ((fs_points[:, 0] - motion_x[i_step]) ** 2 + (fs_points[:, 1] - motion_y[i_step]) ** 2) ** 0.5
        _snap_points = fs_points[_distances <= snap_distance]

        if len(_snap_points) > 0:
            snap_points.append(_snap_points)

    snap_points = np.concatenate(snap_points)
    snap_points = np.array(list(set(map(tuple, snap_points))))

    # print("snap_distance", snap_distance)
    # print("snap_points.shape", snap_points.shape)
    ##################################################################################
    ######################### OPTIMIZE ###############################################
    if len(snap_points) > 0:  # some points are within snapping range

        def loss(params):
            # print(params.shape)
            _d_xs = np.cumsum(params[: num_elements - 1] * np.cos(
                params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
            _d_ys = np.cumsum(params[: num_elements - 1] * np.sin(
                params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
            _xs = np.hstack([params[2 * num_elements - 1] / SCALE_XY0,
                             params[2 * num_elements - 1] / SCALE_XY0 + _d_xs])
            _ys = np.hstack([params[2 * num_elements] / SCALE_XY0,
                             params[2 * num_elements] / SCALE_XY0 + _d_ys])

            distances2 = (snap_points[:, 0].reshape(-1, 1) - _xs.reshape(1, -1)) ** 2 + (
                        snap_points[:, 1].reshape(-1, 1) - _ys.reshape(1, -1)) ** 2
            _loss = np.sum(np.amin(distances2, axis=0))  # near-field

            return _loss / num_elements

        results = minimize(fun=loss,
                           x0=initial_guess,
                           bounds=bounds,
                           options={'maxcor': 30, 'ftol': 1e-08, 'gtol': 1e-07, 'maxfun': 20000, 'maxiter': 20000,
                                    'maxls': 30},
                           method="L-BFGS-B")

        params = results.x

        _d_xs = np.cumsum(params[: num_elements - 1] * np.cos(
            params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
        _d_ys = np.cumsum(params[: num_elements - 1] * np.sin(
            params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
        _xs = np.hstack([params[2 * num_elements - 1] / SCALE_XY0,
                         params[2 * num_elements - 1] / SCALE_XY0 + _d_xs])
        _ys = np.hstack([params[2 * num_elements] / SCALE_XY0,
                         params[2 * num_elements] / SCALE_XY0 + _d_ys])

        if is_reversed:
            return _xs[::-1], _ys[::-1], results.fun, results.success, results.message
        else:
            return _xs, _ys, results.fun, results.success, results.message
    else:
        return motion_x, motion_y, -1, False, ""

In [7]:
def optimize_trajectory_fs_v3(motion_x, motion_y, motion_t,
                              fs_points,
                              leaked_record,
                              snap_range, # +-snap/start_point shift range in fraction of x/y bounding box (max-min)
                              step_range, angle_range, angle0_range):  # +-step_range in fraction of step length; +-angle_range in fraction of Pi
    
    # REVERSE START <-> END POINTS?
    is_start_semivalid = leaked_record["start_x"] > 0 and leaked_record["start_delay"] > 0 
    is_end_semivalid = leaked_record["end_x"] > 0 and leaked_record["end_delay"] > 0
    
    if is_start_semivalid and is_end_semivalid:
        is_reversed = leaked_record["start_delay"] > leaked_record["end_delay"]
    elif is_end_semivalid:
        is_reversed = True
    else:
        is_reversed = False
        
    leaked_delay = leaked_record["end_delay"] if is_reversed else leaked_record["start_delay"]
    leaked_x =  leaked_record["end_x"] if is_reversed else leaked_record["start_x"]
    leaked_y =  leaked_record["end_y"] if is_reversed else leaked_record["start_y"]

    if is_reversed:
        motion_x = motion_x[::-1]
        motion_y = motion_y[::-1]
        motion_t = motion_t[::-1]
    ###################################################################
    num_elements = motion_t.shape[0]
    trajectory_extent = ((motion_x.max()-motion_x.min())**2 + (motion_y.max()-motion_y.min())**2)**0.5
    trajectory_extent_x = abs(motion_x.max()-motion_x.min())   
    trajectory_extent_y = abs(motion_y.max()-motion_y.min())
    
    snap_distance = snap_range * trajectory_extent
    snap_distance_x = snap_range * trajectory_extent_x
    snap_distance_y = snap_range * trajectory_extent_y
    
    time_deltas = np.abs(np.diff(motion_t))  # size "n-1"
    motion_dx = np.diff(motion_x)
    motion_dy = np.diff(motion_y)

    motion_steps_lengths_input = (motion_dx**2 + motion_dy**2)**0.5
    motion_angles_input = np.arctan2(motion_dy, motion_dx) 
    ################  INITIAL/BOUNDARIES VALUES   ##################################      
    angle0_ini = np.array([motion_angles_input[0]])
    angle0_min = np.minimum(motion_angles_input[0]-angle0_range*np.pi, motion_angles_input[0]+angle0_range*np.pi)-0.0000001
    angle0_max = np.maximum(motion_angles_input[0]-angle0_range*np.pi, motion_angles_input[0]+angle0_range*np.pi)+0.0000001
    # DELTA OF ANGLE CHANGE (in abs units) (n-1) segments-1 => n-2
    angles_ini = np.diff(motion_angles_input) 
    angles_min = np.minimum(angles_ini * (1-angle_range), angles_ini * (1+angle_range))
    angles_max = np.maximum(angles_ini * (1-angle_range), angles_ini * (1+angle_range))
    
    step_lengths_ini = np.minimum(time_deltas * V_LIMIT, motion_steps_lengths_input)   
    step_lengths_min = np.minimum(time_deltas * V_LIMIT * 0.99, motion_steps_lengths_input*(1-step_range))
    step_lengths_max = np.minimum(time_deltas * V_LIMIT * 1.00, motion_steps_lengths_input*(1+step_range))  # not more than maximum speed nor allowed variation of step length
    
    initial_start = np.array([motion_x[0], motion_y[0]])
    x_start_min = initial_start[0] - snap_distance_x
    x_start_max = initial_start[0] + snap_distance_x
    y_start_min = initial_start[1] - snap_distance_y
    y_start_max = initial_start[1] + snap_distance_y
    ############# Limit snap range by leaked data (if narrower and leaked data valid)  #######
    if leaked_x > 0 and leaked_delay > 0 and (V_LIMIT * leaked_delay) < trajectory_extent * INVALID_RANGE:

        _x_start_min = max(leaked_x - V_LIMIT * leaked_delay, x_start_min)
        _x_start_max = min(leaked_x + V_LIMIT * leaked_delay, x_start_max)
        _y_start_min = max(leaked_y - V_LIMIT * leaked_delay, y_start_min)
        _y_start_max = min(leaked_y + V_LIMIT * leaked_delay, y_start_max)
        
        x_start_min, x_start_max = (_x_start_min, _x_start_max) if _x_start_min < _x_start_max else (x_start_min, x_start_max)
        y_start_min, y_start_max = (_y_start_min, _y_start_max) if _y_start_min < _y_start_max else (y_start_min, y_start_max)
        
        #initial_start = np.array([(x_start_min+x_start_max)/2, (y_start_min+y_start_max)/2])
    ################ RESCALE XY0/INI-BOUNDS  ####################
    initial_start *= SCALE_XY0
    x_start_min *= SCALE_XY0 
    x_start_max *= SCALE_XY0
    y_start_min *= SCALE_XY0
    y_start_max *= SCALE_XY0
    ################################################################# 
    bounds_angle0 = [(None, None)]#[(angle0_min, angle0_max)]
    bounds_start = [(0, None), (0, None)]
    #bounds_start = [(x_start_min, x_start_max), (y_start_min, y_start_max)]  # 2x pairs
    bounds_angles = [(min_el, max_el) for min_el, max_el in zip(angles_min, angles_max)] # (n-2)x pairs
    bounds_steps = [(min_el, max_el) for min_el, max_el in zip(step_lengths_min, step_lengths_max)] # (n-1)x pairs
    
    initial_guess =  np.concatenate([step_lengths_ini, angles_ini, angle0_ini, initial_start])# (2*n+2)x
    bounds = bounds_steps + bounds_angles + bounds_angle0 + bounds_start # (2*n+2)x pairs
    
    #print(initial_guess)
    #print(bounds)
    
    # :num_elements-1 => step_lengths (n-1)
    # num_elements-1:2*num_elements-3 => angle_changes (n-2)
    # 2*num_elements-3 => angle0 (1) 
    # 2*num_elements-2 => x0 (1)
    # 2*num_elements-1 => y0 (1)
        
    ###############################################################################
    #########################  SNAPPING POINTS   ##################################
    snap_points = []
    
    for i_step, _ in enumerate(motion_x):
        _distances = ((fs_points[:, 0] - motion_x[i_step])**2 + (fs_points[:, 1] - motion_y[i_step])**2)**0.5
        _snap_points = fs_points[_distances <= snap_distance]
        
        if len(_snap_points) > 0:
            snap_points.append(_snap_points)
                
    snap_points = np.concatenate(snap_points)
    snap_points = np.array(list(set(map(tuple, snap_points))))
    
    #print("snap_distance", snap_distance)
    #print("snap_points.shape", snap_points.shape)
    ##################################################################################
    ######################### OPTIMIZE ###############################################
    if len(snap_points) > 0:  # some points are within snapping range
               
        def loss(params):   
            #print(params.shape)
            _d_as = np.cumsum(params[num_elements-1:2*num_elements-3])
            _as = np.hstack([params[2*num_elements-3],
                             params[2*num_elements-3] + _d_as])
            
            _d_xs = np.cumsum(params[:num_elements-1] * np.cos(_as))
            _d_ys = np.cumsum(params[:num_elements-1] * np.sin(_as))

            _xs = np.hstack([params[2*num_elements-2]/SCALE_XY0, 
                             params[2*num_elements-2]/SCALE_XY0 + _d_xs])
            _ys = np.hstack([params[2*num_elements-1]/SCALE_XY0, 
                             params[2*num_elements-1]/SCALE_XY0 + _d_ys])
            #print("_xs.shape", _xs.shape)
            distances2 = (snap_points[:, 0].reshape(-1,1) - _xs.reshape(1,-1))**2 + (snap_points[:, 1].reshape(-1,1) - _ys.reshape(1,-1))**2
            _loss = np.sum(np.amin(distances2, axis=0))

            return _loss/num_elements

        results = minimize(fun=loss,
                           x0=initial_guess,
                           bounds=bounds,
                           options={'maxcor': 30, 'ftol': 1e-08, 'gtol': 1e-07, 'maxfun': 20000, 'maxiter': 20000, 'maxls': 30},
                           method="L-BFGS-B")
                              
        params = results.x

        _d_as = np.cumsum(params[num_elements-1:2*num_elements-3])
        _as = np.hstack([params[2*num_elements-3],
                         params[2*num_elements-3] + _d_as])

        _d_xs = np.cumsum(params[:num_elements-1] * np.cos(_as))
        _d_ys = np.cumsum(params[:num_elements-1] * np.sin(_as))

        _xs = np.hstack([params[2*num_elements-2]/SCALE_XY0, 
                         params[2*num_elements-2]/SCALE_XY0 + _d_xs])
        _ys = np.hstack([params[2*num_elements-1]/SCALE_XY0, 
                         params[2*num_elements-1]/SCALE_XY0 + _d_ys])

        #print("done fitting")
        if is_reversed:
            return _xs[::-1], _ys[::-1], results.fun, results.success, results.message
        else:
            return _xs, _ys, results.fun, results.success, results.message
    else:
        return motion_x, motion_y, -1, False, ""

In [8]:
T_CUT = 200000  # max segement duration in mseconds

def optimize_trajectory_fs_v4(motion_x_full, motion_y_full, motion_t_full,
                              fs_points,
                              leaked_record,
                              snap_range,  # +-snap/start_point shift range in fraction of x/y bounding box (max-min)
                              step_range, angle_range,
                              angle0_range):  # +-step_range in fraction of step length; +-angle_range in fraction of Pi
    
    num_cuts = np.ceil((leaked_record["end_t"] - leaked_record["start_t"])/T_CUT).astype(int)
    num_elements_full = motion_t_full.shape[0]
    
    sum_xs = []
    sum_ys = []
    #print("cuts:", num_cuts)
    for i_cut in range(num_cuts):
        # prepare data for a given segement
        ind_s = i_cut*(num_elements_full//num_cuts)
        ind_e = (num_elements_full//num_cuts)*(i_cut+1 if i_cut < num_cuts-1 else i_cut+2)  # large end index if last segment
        
        motion_x = motion_x_full[ind_s:ind_e]
        motion_y = motion_y_full[ind_s:ind_e] 
        motion_t = motion_t_full[ind_s:ind_e]
        
        #print(f"segment/full: {motion_x.shape}/{motion_x_full.shape}")
        ###################################################################
        num_elements = motion_t.shape[0]
        trajectory_extent = ((motion_x.max() - motion_x.min()) ** 2 + (motion_y.max() - motion_y.min()) ** 2) ** 0.5
        trajectory_extent_x = abs(motion_x.max() - motion_x.min())
        trajectory_extent_y = abs(motion_y.max() - motion_y.min())

        snap_distance = snap_range * trajectory_extent
        snap_distance_x = snap_range * trajectory_extent_x
        snap_distance_y = snap_range * trajectory_extent_y

        time_deltas = np.abs(np.diff(motion_t))  # size "n-1"
        motion_dx = np.diff(motion_x)
        motion_dy = np.diff(motion_y)

        motion_steps_lengths_input = (motion_dx ** 2 + motion_dy ** 2) ** 0.5
        motion_angles_input = np.arctan2(motion_dy, motion_dx)
        # print(num_elements)
        ################  INITIAL/BOUNDARIES VALUES   ##################################
        step_lengths_ini = np.amin(np.vstack([time_deltas * V_LIMIT, motion_steps_lengths_input]), axis=0)
        angles_ini = motion_angles_input

        step_lengths_min = np.amin(np.vstack([time_deltas * V_LIMIT * 0.99, motion_steps_lengths_input * (1 - step_range)]),
                                   axis=0)
        step_lengths_max = np.amin(np.vstack([time_deltas * V_LIMIT * 1.00, motion_steps_lengths_input * (1 + step_range)]),
                                   axis=0)  # not more than maximum speed nor allowed variation of step length
        angles_min = motion_angles_input - np.pi * angle_range
        angles_max = motion_angles_input + np.pi * angle_range

        angle0_ini = np.array([0.0])

        initial_start = np.array([motion_x[0], motion_y[0]])

        x_start_min = initial_start[0] - snap_distance_x
        x_start_max = initial_start[0] + snap_distance_x
        y_start_min = initial_start[1] - snap_distance_y
        y_start_max = initial_start[1] + snap_distance_y

        ################ RESCALE XY0/INI-BOUNDS  ####################
        initial_start *= SCALE_XY0
        x_start_min *= SCALE_XY0
        x_start_max *= SCALE_XY0
        y_start_min *= SCALE_XY0
        y_start_max *= SCALE_XY0
        #################################################################
        bounds_angle0 = [(-angle0_range * np.pi, angle0_range * np.pi)]
        bounds_start = [(0, None), (0, None)]
        #bounds_start = [(x_start_min, x_start_max), (y_start_min, y_start_max)]  # 2x pairs
        bounds_steps = [(min_el, max_el) for min_el, max_el in zip(step_lengths_min, step_lengths_max)]  # (n-1)x pairs
        bounds_angles = [(min_el, max_el) for min_el, max_el in zip(angles_min, angles_max)]  # (n-1)x pairs

        initial_guess = np.concatenate([step_lengths_ini, angles_ini, angle0_ini, initial_start])  # (2*n+2)x
        bounds = bounds_steps + bounds_angles + bounds_angle0 + bounds_start  # (2*n+2)x pairs
        ###############################################################################
        #########################  SNAPPING POINTS   ##################################
        snap_points = []

        for i_step, _ in enumerate(motion_x):
            _distances = ((fs_points[:, 0] - motion_x[i_step]) ** 2 + (fs_points[:, 1] - motion_y[i_step]) ** 2) ** 0.5
            _snap_points = fs_points[_distances <= snap_distance]

            if len(_snap_points) > 0:
                snap_points.append(_snap_points)



        # print("snap_distance", snap_distance)
        # print("snap_points.shape", snap_points.shape)
        ##################################################################################
        ######################### OPTIMIZE ###############################################
        if len(snap_points) > 0:  # some points are within snapping range
            
            snap_points = np.concatenate(snap_points)
            snap_points = np.array(list(set(map(tuple, snap_points))))

            def loss(params):
                # print(params.shape)
                _d_xs = np.cumsum(params[: num_elements - 1] * np.cos(
                    params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
                _d_ys = np.cumsum(params[: num_elements - 1] * np.sin(
                    params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
                _xs = np.hstack([params[2 * num_elements - 1] / SCALE_XY0,
                                 params[2 * num_elements - 1] / SCALE_XY0 + _d_xs])
                _ys = np.hstack([params[2 * num_elements] / SCALE_XY0,
                                 params[2 * num_elements] / SCALE_XY0 + _d_ys])

                distances2 = (snap_points[:, 0].reshape(-1, 1) - _xs.reshape(1, -1)) ** 2 + (
                            snap_points[:, 1].reshape(-1, 1) - _ys.reshape(1, -1)) ** 2
                _loss = np.sum(np.amin(distances2, axis=0))  # near-field

                return _loss / num_elements

            results = minimize(fun=loss,
                               x0=initial_guess,
                               bounds=bounds,
                               options={'maxcor': 30, 'ftol': 1e-08, 'gtol': 1e-07, 'maxfun': 20000, 'maxiter': 20000,
                                        'maxls': 30},
                               method="L-BFGS-B")

            params = results.x

            _d_xs = np.cumsum(params[: num_elements - 1] * np.cos(
                params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
            _d_ys = np.cumsum(params[: num_elements - 1] * np.sin(
                params[2 * num_elements - 2] + params[num_elements - 1: 2 * num_elements - 2]))
            _xs = np.hstack([params[2 * num_elements - 1] / SCALE_XY0,
                             params[2 * num_elements - 1] / SCALE_XY0 + _d_xs])
            _ys = np.hstack([params[2 * num_elements] / SCALE_XY0,
                             params[2 * num_elements] / SCALE_XY0 + _d_ys])

            sum_xs.append(_xs)
            sum_ys.append(_ys)
        else:
            sum_xs.append(motion_x)
            sum_ys.append(motion_y)

    return np.concatenate(sum_xs), np.concatenate(sum_ys), -1, False, ""

In [13]:
############################################################################################################
def snap2fs(predicted_data, fs_data, leaked_data, snap_range, step_range, angle_range, angle0_range):

    snapped_data = {}
    site_ids = ["5d2709d403f801723c32bd39"]
    # long traj=> ["5d27096c03f801723c31e5e0", "5d2709b303f801723c327472", "5d2709c303f801723c3299ee", 
                 # "5d2709d403f801723c32bd39", "5da138274db8ce0c98bbd3d2", "5da1382d4db8ce0c98bbe92e", 
                 # "5da138764db8ce0c98bcaa46", "5da958dd46f8266d0737457b", "5dbc1d84c1eb61796cf7c010"]
    
    #site_ids = ["5d27096c03f801723c31e5e0", "5d2709b303f801723c327472", "5d2709c303f801723c3299ee",
    #            "5d2709d403f801723c32bd39", "5da138274db8ce0c98bbd3d2", "5da1382d4db8ce0c98bbe92e",
     #           "5da138764db8ce0c98bcaa46", "5da958dd46f8266d0737457b", "5dbc1d84c1eb61796cf7c010"]
    #site_ids = ["5dc8cea7659e181adb076a3f"]#, "5da138754db8ce0c98bca82f", "5d2709e003f801723c32d896", "5da138364db8ce0c98bc00f1"]
    #site_ids = predicted_data.keys()
    for site_id in tqdm(site_ids):#
        
        #print(site_id)
        snapped_data[site_id] = {}
        for trace_id in tqdm(predicted_data[site_id]):  # over traces
            #print(trace_id)
            predicted_record = predicted_data[site_id][trace_id][["time", "x", "y"]].copy(deep=True)
            _floor = predicted_data[site_id][trace_id]["floor"][0]
            fs_record = fs_data[site_id][_floor]
            leaked_record = leaked_data[site_id][trace_id]
            
            _x, _y, _fun, _isOk, _exit_msg = optimize_trajectory_fs_v4(predicted_record.x.to_numpy(),
                                                                       predicted_record.y.to_numpy(),
                                                                       predicted_record.time.to_numpy(),
                                                                       fs_record,
                                                                       leaked_record,
                                                                       snap_range, step_range, angle_range, angle0_range)
            #print(f"{_exit_msg}: {_isOk}")
            #if not _isOk:
             #   print(f"Failed Site/TraceIds: {site_id}/{trace_id}")
            
            snapped_data[site_id][trace_id] = pd.DataFrame(_x, columns=["x"])
            snapped_data[site_id][trace_id]["y"] = _y
            snapped_data[site_id][trace_id]["floor"] = _floor
            snapped_data[site_id][trace_id]["time"] = predicted_record.time.to_numpy()
            
            #print(_fun)
            #break
        break
        
    return snapped_data

###############    MULTIPROCESSING VERSION     #################################################################
def snap2fs_multi(predicted_data, fs_data, leaked_data, snap_range, step_range, angle_range, angle0_range):

    #site_ids = ["5da1389e4db8ce0c98bd0547", "5d27075f03f801723c2e360f"]#, "5d2709b303f801723c327472", "5d27097f03f801723c320d97", "5da138b74db8ce0c98bd4774", "5d2709d403f801723c32bd39"]
    
    site_ids = ["5d27096c03f801723c31e5e0", "5d2709b303f801723c327472", "5d2709c303f801723c3299ee",
                "5d2709d403f801723c32bd39", "5da138274db8ce0c98bbd3d2", "5da1382d4db8ce0c98bbe92e",
                "5da138764db8ce0c98bcaa46", "5da958dd46f8266d0737457b", "5dbc1d84c1eb61796cf7c010"]
    site_ids = predicted_data.keys()
    
    input_data = [(site_id, predicted_data[site_id], fs_data[site_id], leaked_data[site_id], snap_range, step_range, angle_range, angle0_range) for site_id in site_ids]
    
    pool = Pool(cpu_count()-3) # Create a multiprocessing Pool
    snapped_data = pool.starmap(fs_multi, input_data)  # process input_data iterable with pool

    pool.close()
    pool.join()
        
    res_d = snapped_data[0]
    for i, _d in enumerate(snapped_data):
        if i>0:
            res_d.update(_d)
        
    return res_d

def snap2fs_multiF(predicted_data, fs_data, leaked_data, snap_range, step_range, angle_range, angle0_range):

    #site_ids = ["5da1389e4db8ce0c98bd0547", "5d27075f03f801723c2e360f"]#, "5d2709b303f801723c327472", "5d27097f03f801723c320d97", "5da138b74db8ce0c98bd4774", "5d2709d403f801723c32bd39"]
    
    site_ids = ["5d27096c03f801723c31e5e0", "5d2709b303f801723c327472", "5d2709c303f801723c3299ee",
                "5d2709d403f801723c32bd39", "5da138274db8ce0c98bbd3d2", "5da1382d4db8ce0c98bbe92e",
                "5da138764db8ce0c98bcaa46", "5da958dd46f8266d0737457b", "5dbc1d84c1eb61796cf7c010"]
    site_ids = predicted_data.keys()
    
    input_data = [(site_id, predicted_data[site_id], fs_data[site_id], leaked_data[site_id], snap_range, step_range, angle_range, angle0_range) for site_id in site_ids]
    
    pool = Pool(cpu_count()-3) # Create a multiprocessing Pool
    snapped_data = pool.starmap(fs_multiF, input_data)  # process input_data iterable with pool

    pool.close()
    pool.join()
        
    res_d = snapped_data[0]
    for i, _d in enumerate(snapped_data):
        if i>0:
            res_d.update(_d)
        
    return res_d

In [10]:
model_name = "blend_x7_0305"

predicted_data = pickle.load(open(f"./submit/fit_data/post/{model_name}_snapped2motion2s.pkl", "rb")) #
fs_data = pickle.load(open("./data_out/freespace_1m_siteid_floorid.pkl", "rb"))
leaked_data = pickle.load(open("./data_out/leaked/leaked_siteid_traceid_all_delays.pkl", "rb")) 

snapped2fs = snap2fs_multi(predicted_data, fs_data, leaked_data, snap_range=0.2, step_range=0.4, angle_range=0.2, angle0_range=0.2)
plot_predictions_multi(model_name, snapped2fs, sufix=f"snap2motion2s-fsV2nlB-02-04-02-02")
with open(f"./submit/fit_data/post/{model_name}_snapped2motion2s-fsV2nlB-02-04-02-02.pkl", "wb") as f:
    pickle.dump(snapped2fs, f)

  0%|          | 0/24 [00:00<?, ?it/s]

In [14]:
predicted_data = pickle.load(open(f"./submit/fit_data/post/{model_name}_snapped2motion4s.pkl", "rb")) #
snapped2fs = snap2fs_multiF(predicted_data, fs_data, leaked_data, snap_range=0.2, step_range=0.4, angle_range=0.2, angle0_range=0.2)
plot_predictions_multi(model_name, snapped2fs, sufix=f"snap2motion4s-fsV2nlF-02-04-02-02")
with open(f"./submit/fit_data/post/{model_name}_snapped2motion4s-fsV2nlF-02-04-02-02.pkl", "wb") as f:
    pickle.dump(snapped2fs, f)

  0%|          | 0/24 [00:00<?, ?it/s]

In [15]:
predicted_data = pickle.load(open(f"./submit/fit_data/post/{model_name}_snapped2motion3s.pkl", "rb")) #
snapped2fs = snap2fs_multiF(predicted_data, fs_data, leaked_data, snap_range=0.2, step_range=0.4, angle_range=0.2, angle0_range=0.2)
plot_predictions_multi(model_name, snapped2fs, sufix=f"snap2motion3s-fsV2nlF-02-04-02-02")
with open(f"./submit/fit_data/post/{model_name}_snapped2motion3s-fsV2nlF-02-04-02-02.pkl", "wb") as f:
    pickle.dump(snapped2fs, f)

  0%|          | 0/24 [00:00<?, ?it/s]

In [16]:
predicted_data = pickle.load(open(f"./submit/fit_data/post/{model_name}_snapped2motion2s.pkl", "rb")) #
snapped2fs = snap2fs_multiF(predicted_data, fs_data, leaked_data, snap_range=0.2, step_range=0.4, angle_range=0.2, angle0_range=0.2)
plot_predictions_multi(model_name, snapped2fs, sufix=f"snap2motion2s-fsV2nlF-02-04-02-02")
with open(f"./submit/fit_data/post/{model_name}_snapped2motion2s-fsV2nlF-02-04-02-02.pkl", "wb") as f:
    pickle.dump(snapped2fs, f)

  0%|          | 0/24 [00:00<?, ?it/s]

---------------------------------------------------------

Final snapping to grid/waypoints -> plotting -> submission

In [21]:
model_name = "blend_x7_0305"
fork_name = "2motion4-fsV2nlF-02-04-02-01"

predicted_data_post = pickle.load(open(f"./submit/fit_data/post/{model_name}_snapped{fork_name}.pkl", "rb")) #
grid_data = pickle.load(open("./data_out/waypoints_siteid_floorid.pkl", "rb"))#pickle.load(open("./data_out/freespace_1m_siteid_floorid.pkl", "rb"))  # pickle.load(open("./data_out/waypoints_siteid_floorid.pkl", "rb"))
timestamps = pickle.load(open(f"./data_out/submission_timestamps_traceid.pkl", "rb"))
aux_grid_data = pickle.load(open("./data_out/freespace_2m_siteid_floorid.pkl", "rb"))

snapped2grid = snap2grid(predicted_data_post, grid_data, aux_grid_data, timestamps, snap_range=4, aux_snap_range=50)

make_submission(model_name, snapped2grid, sufix=f"snap{fork_name}-gridWPs4")

  0%|          | 0/24 [00:00<?, ?it/s]

In [15]:
#plot_predictions_multi(model_name, snapped2grid, sufix=f"snap2motion-gridWPs5")
plot_predictions_multi(model_name, snapped2grid, sufix=f"snap2motion-fsV2p05-02-03-01-01-gridWPs5")

  0%|          | 0/24 [00:00<?, ?it/s]

In [25]:
model_name = "blend_x7_0305"
data_post = pickle.load(open(f"./submit/fit_data/post/blend_x7_0305_snapped2motion4-fsV2nlF-02-04-02-01.pkl", "rb")) #
plot_predictions_multi(model_name, data_post, sufix=f"snap2motion4-fsV2nlF-02-04-02-01")

  0%|          | 0/24 [00:00<?, ?it/s]

In [26]:
model_name = "blend_x7_0305"
data_post = pickle.load(open(f"./submit/fit_data/post/blend_x7_0305_snapped2motion4-fsV2nlB-02-04-02-01.pkl", "rb")) #
plot_predictions_multi(model_name, data_post, sufix=f"snap2motion4-fsV2nlB-02-04-02-01")

  0%|          | 0/24 [00:00<?, ?it/s]